# Test base/niagads/validate_medatata

In [5]:
import subprocess
import json

def pretty_print(resultStr):
    resultJson = json.loads(resultStr.replace('running\n', '').replace('\\"', ''))
    print(json.dumps(resultJson, indent=4))

## script test `../../bases/niagads/validate_metadata/core.py`

In [6]:
# usage 
CMD_ROOT = ['python', '../../bases/niagads/validate_metadata/core.py']
result = subprocess.run(CMD_ROOT + ['--help'], capture_output=True, text=True)
print(result.stderr)
print(result.stdout)


running
usage: core.py [-h] [--template TEMPLATE] --metadataFileType
               {BIOSOURCE_PROPERTIES,FILE_MANIFEST} [--log] [--failOnError]
               [--schemaDir SCHEMADIR]
               [--metadataFilePrefix METADATAFILEPREFIX]
               [--metadataFile METADATAFILE] [--schemaFile SCHEMAFILE]
               [--idField IDFIELD]

validate_metadata This script allows the user validate a sample or file
manifest metadata file arranged in tabular format (field names in columns,
values in rows) against a JSON-Schema file. Results are piped to STDOUT unless
`--log` option is specified. This tool accepts tab separated value files
(.tab) as well as excel (.xls, .xlsx) files. This file can also be imported as
a module and contains the following functions / tyes: * MetadataValidatorType
- enum of types of expected metadata files * initialize_validator - returns an
initialized BiosourcePropertiesValidator or FileManifestValidator *
get_templated_schema_file - builds schema file n

In [7]:
# file manifest test
schemaFile = './schemas/file_manifest.json'
metadataFile = './examples/test_file_manifest.tab'
cmd = ['python', '../bases/niagads/validate_metadata/core.py']
args = ['--metadataFileType', 'file_manifest', '--schemaFile', schemaFile, '--metadataFile', metadataFile]
result = subprocess.run(CMD_ROOT + args, capture_output=True, text=True)
print(result.stderr)
print(result.stdout)



running
{"errors": []}



In [8]:
# file manifest with errors te
schemaFile = './schemas/file_manifest.json'
metadataFile = './examples/test_file_manifest_with_errors.tab'
cmd = ['python', '../bases/niagads/validate_metadata/core.py']
args = ['--metadataFileType', 'file_manifest',  '--schemaFile', schemaFile, '--metadataFile', metadataFile]
result = subprocess.run(CMD_ROOT + args, capture_output=True, text=True)
print(result.stderr)
pretty_print(result.stdout)




{
    "errors": [
        {
            "1": [
                "'c6779ec2960296ed9a8d67f64422' does not match '^[a-fA-F0-9]{32}$'"
            ]
        },
        {
            "6": [
                "required field `sample_id` cannot be empty / null"
            ]
        }
    ]
}


In [9]:
# templated file manifest
schemaDir = 'schemas'
pattern = 'examples/test_'
cmd = ['python', '../bases/niagads/validate_metadata/core.py']
args = ['--metadataFileType', 'file_manifest', '--template', 'file_manifest', '--schemaDir', schemaDir, '--metadataFilePrefix', pattern]
result = subprocess.run(CMD_ROOT + args, capture_output=True, text=True)
print(result.stderr)
pretty_print(result.stdout)




{
    "errors": []
}


In [10]:
# templated participant info file
schemaDir = 'schemas'
pattern = 'examples/test_'
idField = 'participant_id'
cmd = ['python', '../bases/niagads/validate_metadata/core.py']
args = ['--metadataFileType', 'biosource_properties', 
    '--template', 'participant_info', 
    '--schemaDir', schemaDir, 
    '--metadataFilePrefix', pattern,
    '--idField', idField
    ]
result = subprocess.run(CMD_ROOT + args, capture_output=True, text=True)
print(result.stderr)
pretty_print(result.stdout)




{
    "errors": []
}


## Test package

In [11]:
import niagads.validate_metadata.core as vm

schemaDir = 'schemas'
pattern = 'examples/test_'
idField = 'participant_id'
template = 'participant_info'

# get files from template and path variables
schemaFile = vm.get_templated_schema_file(schemaDir, template)
print(f'Schema File: {schemaFile}')

metadataFile = vm.get_templated_metadata_file(pattern, template)
print(f'Metadata File: {metadataFile}')

# straight run
validationResult = vm.run(metadataFile, schemaFile, 'biosource_properties', idField)
print(f'Straight Run result: {validationResult}')

# get an initialized validator object
validator = vm.initialize_validator(metadataFile, schemaFile, 'biosource_validator', idField)
print(f'Validator type: {type(validator)}')
print(f'Schema: {validator.get_schema()}')
print(f'Parsed Metadata: {json.dumps(validator.get_metadata(), indent=4)}')
print(f'Biosource IDs: {validator.get_biosource_ids()}')
print(f'Race: {validator.get_field_values('race')}')
validationResult = validator.run()
print(f'Validation Result: {validationResult}')

Schema File: schemas/participant_info.json
Metadata File: examples/test_participant_info.tab
Straight Run result: {'errors': []}


ValueError: Invalid `metadataType` : 'biosource_validator'.  Valid values are: BIOSOURCE_PROPERTIES, FILE_MANIFEST

In [ ]:
# dss validation
schemaDir = './schemas'
filePrefix = './examples/test_'
cmd = ['python', '../dss_metadata_validator.py']
args = ['--schemaDir', schemaDir, '--metadataFilePrefix', filePrefix]
result = subprocess.run(cmd + args, capture_output=True, text=True)
print(result.stderr)
print(result.stdout)
pretty_print(result.stdout)

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.

{"participant_info": {"errors": []}, "sample_info": {"errors": [{"invalid_PARTICIPANT_ID": ["DONOR7"]}], "warnings": [{"missing_PARTICIPANT_ID": ["DONOR5"]}]}, "file_manifest": {"errors": []}}

{
    "participant_info": {
        "errors": []
    },
    "sample_info": {
        "errors": [
            {
                "invalid_PARTICIPANT_ID": [
                    "DONOR7"
                ]
            }
        ],
        "warnings": [
            {
                "missing_PARTICIPANT_ID": [
                    "DONOR5"
                ]
            }
        ]
    },
    "file_manifest": {
        "errors": []
    }
}
